# Amazon Bedrock AgentCore 런타임(Runtime)에서 MCP 서버 호스팅 - AWS IAM 인바운드 인증(Authentication)

## 개요

이 튜토리얼에서는 Amazon Bedrock AgentCore 런타임(Runtime)에서 MCP(Model Context Protocol) 서버를 호스팅하는 방법을 배웁니다. Amazon Bedrock AgentCore Python SDK를 사용하여 MCP 도구(Tool)를 Amazon Bedrock AgentCore와 호환되는 MCP 서버로 래핑합니다.

Amazon Bedrock AgentCore Python SDK는 MCP 서버 구현 세부사항을 처리하므로 도구(Tool)의 핵심 기능에 집중할 수 있습니다. 코드를 직접 통신을 위한 AgentCore 표준화된 MCP 프로토콜 계약으로 변환합니다.

[MCP 프로토콜](https://modelcontextprotocol.io/docs/getting-started/intro) 사양은 전통적으로 인증(Authentication)에 OAuth 토큰을 요구하지만, AgentCore 런타임(Runtime)은 MCP 서버에 대한 인바운드 요청에 AWS IAM 자격 증명을 구성(Configuration)할 수 있는 기능을 제공하여 중요한 기업 요구사항을 해결합니다.

### 튜토리얼 세부사항

| 정보                | 세부사항                                                   |
|:--------------------|:----------------------------------------------------------|
| 튜토리얼 유형       | 도구(Tool) 호스팅                                          |
| 도구(Tool) 유형     | MCP 서버                                                   |
| 튜토리얼 구성 요소   | AgentCore 런타임(Runtime)에 MCP 서버 호스팅                |
| 튜토리얼 분야       | 범용                                                       |
| 예제 복잡도         | 쉬움                                                       |
| 사용 SDK            | Amazon BedrockAgentCore Python SDK 및 MCP                  |

### 튜토리얼 아키텍처

이 튜토리얼에서는 MCP 서버를 AgentCore 런타임(Runtime)에 배포(Deployment)하는 방법을 설명합니다.

데모 목적으로 `add_numbers`, `multiply_numbers`, `greet_user` 3가지 도구(Tool)가 있는 간단한 MCP 서버를 사용합니다.

<div style="text-align:left">
    <img src="images/hosting_mcp_server.png" width="60%"/>
</div>

### 튜토리얼 주요 기능

* 커스텀 도구(Tool)로 MCP 서버 생성
* 로컬에서 MCP 서버 테스트
* Amazon Bedrock AgentCore 런타임(Runtime)에 MCP 서버 호스팅
* 인증(Authentication)으로 배포(Deployment)된 MCP 서버 호출(Invocation)

## 사전 요구사항

이 튜토리얼을 실행하려면 다음이 필요합니다:
* Python 3.10+
* AWS 자격 증명 구성(Configuration)
* Amazon Bedrock AgentCore SDK
* MCP(Model Context Protocol) 라이브러리
* Docker 데몬 실행 중

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from bedrock_agentcore_starter_toolkit.operations.runtime import destroy_bedrock_agentcore
from boto3.session import Session
from pathlib import Path
import os

In [ ]:
boto_session = Session()
region = boto_session.region_name

agentcore_control_client = boto_session.client("bedrock-agentcore-control", region_name=region)
ssm_client = boto_session.client('ssm', region_name=region)

tool_name = "mcp_server_iam"

## Understanding MCP (Model Context Protocol)

MCP is a protocol that allows AI models to securely access external data and tools. Key concepts:

* **Tools**: Functions that the AI can call to perform actions
* **Streamable HTTP**: Transport protocol used by AgentCore Runtime
* **Session Isolation**: Each client gets isolated sessions via `Mcp-Session-Id` header
* **Stateless Operation**: Servers must support stateless operation for scalability

AgentCore Runtime expects MCP servers to be hosted on `0.0.0.0:8000/mcp` as the default path.

### Project Structure

Let's set up our project with the proper structure:

```
mcp_server_project/
├── mcp_server.py              # Main MCP server code
├── mcp_client.py          # Local testing client
├── mcp_client_remote.py   # Remote testing client
├── requirements.txt          # Dependencies
└── __init__.py              # Python package marker
```

## Creating MCP Server

Now let's create our MCP server with three simple tools. The server uses FastMCP with `stateless_http=True` which is required for AgentCore Runtime compatibility.

In [ ]:
%%writefile mcp_server.py
from mcp.server.fastmcp import FastMCP
from starlette.responses import JSONResponse

mcp = FastMCP(host="0.0.0.0", stateless_http=True)

@mcp.tool()
def add_numbers(a: int, b: int) -> int:
    """Add two numbers together"""
    return a + b

@mcp.tool()
def multiply_numbers(a: int, b: int) -> int:
    """Multiply two numbers together"""
    return a * b

@mcp.tool()
def greet_user(name: str) -> str:
    """Greet a user by name"""
    return f"Hello, {name}! Nice to meet you."

if __name__ == "__main__":
    mcp.run(transport="streamable-http")

## MCP(Model Context Protocol) 이해하기

MCP는 AI 모델이 외부 데이터와 도구(Tool)에 안전하게 액세스할 수 있게 해주는 프로토콜입니다. 주요 개념:

* **도구(Tool)**: AI가 작업을 수행하기 위해 호출할 수 있는 함수
* **스트리밍 가능한 HTTP**: AgentCore 런타임(Runtime)에서 사용하는 전송 프로토콜
* **세션 격리**: 각 클라이언트는 `Mcp-Session-Id` 헤더를 통해 격리된 세션을 받음
* **무상태 운영**: 서버는 확장성을 위해 무상태 운영을 지원해야 함

AgentCore 런타임(Runtime)은 MCP 서버가 기본 경로로 `0.0.0.0:8000/mcp`에서 호스팅될 것으로 예상합니다.

### 프로젝트 구조

적절한 구조로 프로젝트를 설정해 보겠습니다:

```
mcp_server_project/
├── mcp_server.py              # 메인 MCP 서버 코드
├── mcp_client.py          # 로컬 테스트 클라이언트
├── mcp_client_remote.py   # 원격 테스트 클라이언트
├── requirements.txt          # 종속성
└── __init__.py              # Python 패키지 마커
```

## MCP 서버 생성

이제 세 가지 간단한 도구(Tool)로 MCP 서버를 생성해 보겠습니다. 서버는 AgentCore 런타임(Runtime) 호환성을 위해 필요한 `stateless_http=True`로 FastMCP를 사용합니다.

In [ ]:
%%writefile mcp_client.py
import asyncio

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

async def main():
    mcp_url = "http://localhost:8000/mcp"
    headers = {}

    async with streamablehttp_client(mcp_url, headers, timeout=120, terminate_on_close=False) as (
        read_stream,
        write_stream,
        _,
    ):
        async with ClientSession(read_stream, write_stream) as session:
            await session.initialize()
            tool_result = await session.list_tools()
            print("Available tools:")
            for tool in tool_result.tools:
                print(f"  - {tool.name}: {tool.description}")

if __name__ == "__main__":
    asyncio.run(main())

 ### Testing Locally

To test your MCP server locally:

1. **Terminal 1**: Start the MCP server
   ```bash
   python mcp_server.py
   ```
   
2. **Terminal 2**: Run the test client
   ```bash
   python mcp_client.py
   ```

You should see your three tools listed in the output.

### 이 코드가 하는 일

* **FastMCP**: 도구(Tool)를 호스팅할 수 있는 MCP 서버 생성
* **@mcp.tool()**: Python 함수를 MCP 도구(Tool)로 변환하는 데코레이터
* **stateless_http=True**: AgentCore 런타임(Runtime) 호환성에 필요
* **도구(Tool)**: 다양한 유형의 작업을 보여주는 세 가지 간단한 도구(Tool)

In [ ]:
print(f"Using AWS region: {region}")

required_files = ["mcp_server.py", "requirements.txt"]
for file in required_files:
    if not os.path.exists(file):
        raise FileNotFoundError(f"Required file {file} not found")
print("All required files found ✓")

agentcore_runtime = Runtime()

print("Configuring AgentCore Runtime...")
response = agentcore_runtime.configure(
    entrypoint="mcp_server.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    protocol="MCP",
    agent_name=tool_name,
)
print("Configuration completed ✓")

## 로컬 테스트 클라이언트 생성

AgentCore 런타임(Runtime)에 배포(Deployment)하기 전에 로컬에서 MCP 서버를 테스트할 클라이언트를 생성해 보겠습니다:

In [ ]:
print("Launching MCP server to AgentCore Runtime...")
print("This may take several minutes...")
launch_result = agentcore_runtime.launch()
print("Launch completed ✓")
print(f"Agent ARN: {launch_result.agent_arn}")
print(f"Agent ID: {launch_result.agent_id}")

In [ ]:

agent_arn_response = ssm_client.put_parameter(
    Name='/mcp_server/runtime_iam/agent_arn',
    Value=launch_result.agent_arn,
    Type='String',
    Description='Agent ARN for MCP server with inbound auth',
    Overwrite=True
)
print("✓ Agent ARN stored in Parameter Store")

print("\nConfiguration stored successfully!")
print(f"Agent ARN: {launch_result.agent_arn}")

## Creating Remote Testing Client

Now let's create a client to test our deployed MCP server. This client will retrieve the necessary credentials from AWS and connect to the deployed server:

In [ ]:
%%writefile mcp_client_remote.py       
import asyncio
import sys
import logging
import boto3
from boto3.session import Session
from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client
from streamable_http_sigv4 import streamablehttp_client_with_sigv4


logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)


def create_streamable_http_transport_sigv4(
    mcp_url: str, service_name: str, region: str
):
    """
    Create a streamable HTTP transport with AWS SigV4 authentication.

    This function creates an MCP client transport that uses AWS Signature Version 4 (SigV4)
    to authenticate requests. This is necessary because standard MCP clients don't natively
    support AWS IAM authentication, and this bridges that gap.

    Args:
        mcp_url (str): The URL of the MCP gateway endpoint
        service_name (str): The AWS service name for SigV4 signing (typically "bedrock-agentcore")
        region (str): The AWS region where the gateway is deployed

    Returns:
        StreamableHTTPTransportWithSigV4: A transport instance configured for SigV4 auth

    Example:
        >>> transport = create_streamable_http_transport_sigv4(
        ...     mcp_url=".../mcp",
        ...     service_name="bedrock-agentcore",
        ...     region="us-west-2"
        ... )
    """
    # Get AWS credentials from the current boto3 session
    # These credentials will be used to sign requests with SigV4
    session = boto3.Session()
    credentials = session.get_credentials()

    # Create and return the custom transport with SigV4 signing capability
    return streamablehttp_client_with_sigv4(
        url=mcp_url,
        credentials=credentials,
        service=service_name,
        region=region,
    )


def get_full_tools_list(client):
    """
    Retrieve the complete list of tools from an MCP client, handling pagination.

    MCP servers may return tools in paginated responses. This function handles the
    pagination automatically and returns all available tools in a single list.

    Args:
        client: An MCP client instance (from strands.tools.mcp.mcp_client.MCPClient)

    Returns:
        list: A complete list of all tools available from the MCP server

    Example:
        >>> mcp_client = MCPClient(lambda: create_transport())
        >>> all_tools = get_full_tools_list(mcp_client)
        >>> print(f"Found {len(all_tools)} tools")
    """
    more_tools = True
    tools = []
    pagination_token = None

    # Loop until we've fetched all pages
    while more_tools:
        tmp_tools = client.list_tools_sync(pagination_token=pagination_token)

        tools.extend(tmp_tools)

        # Check if there are more pages to fetch
        if tmp_tools.pagination_token is None:
            # No more pages - we're done
            more_tools = False
        else:
            # More pages exist - prepare to fetch the next one
            more_tools = True
            pagination_token = tmp_tools.pagination_token

    return tools


async def main():
    boto_session = Session()
    region = boto_session.region_name
    print(f"Using AWS region: {region}")

    ssm_client = boto3.client("ssm", region_name=region)

    agent_arn_response = ssm_client.get_parameter(
        Name="/mcp_server/runtime_iam/agent_arn"
    )
    agent_arn = agent_arn_response["Parameter"]["Value"]
    print(f"Retrieved Agent ARN: {agent_arn}")

    if not agent_arn:
        print("❌ Error: AGENT_ARN not found")
        sys.exit(1)

    encoded_arn = agent_arn.replace(":", "%3A").replace("/", "%2F")
    mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"

    try:
        async with create_streamable_http_transport_sigv4(
            mcp_url=mcp_url, service_name="bedrock-agentcore", region=region
        ) as (
            read_stream,
            write_stream,
            _,
        ):
            async with ClientSession(read_stream, write_stream) as session:
                print("\n🔄 Initializing MCP session...")
                await session.initialize()
                print("✓ MCP session initialized")

                print("\n🔄 Listing available tools...")
                tool_result = await session.list_tools()

                print("\n📋 Available MCP Tools:")
                print("=" * 50)
                for tool in tool_result.tools:
                    print(f"🔧 {tool.name}")
                    print(f"   Description: {tool.description}")
                    if hasattr(tool, "inputSchema") and tool.inputSchema:
                        properties = tool.inputSchema.get("properties", {})
                        if properties:
                            print(f"   Parameters: {list(properties.keys())}")
                    print()

                print(f"✅ Successfully connected to MCP server!")
                print(f"Found {len(tool_result.tools)} tools available.")

    except Exception as e:
        print(f"❌ Error connecting to MCP server: {e}")
        import traceback

        print("\n🔍 Full error traceback:")
        traceback.print_exc()
        sys.exit(1)


if __name__ == "__main__":
    asyncio.run(main())


 ### 로컬에서 테스트

MCP 서버를 로컬에서 테스트하려면:

1. **터미널 1**: MCP 서버 시작
   ```bash
   python mcp_server.py
   ```
   
2. **터미널 2**: 테스트 클라이언트 실행
   ```bash
   python mcp_client.py
   ```

출력에서 세 가지 도구(Tool)가 나열되어 있는 것을 볼 수 있습니다.

In [ ]:
print("Testing deployed MCP server...")
print("=" * 50)
!python mcp_client_remote.py

## AgentCore 런타임(Runtime) 배포(Deployment) 구성(Configuration)

다음으로 스타터 툴킷을 사용하여 진입점, 방금 생성한 실행 역할 및 requirements 파일로 AgentCore 런타임(Runtime) 배포(Deployment)를 구성(Configuration)합니다. 또한 시작 시 Amazon ECR 리포지토리를 자동 생성하도록 스타터 키트를 구성(Configuration)합니다.

구성(Configuration) 단계에서 애플리케이션 코드를 기반으로 도커 파일이 생성됩니다.

<div style="text-align:left">
    <img src="images/configure.png" width="60%"/>
</div>

In [ ]:
%%writefile invoke_mcp_tools.py
import asyncio
import sys
import os
import logging
import boto3
from boto3.session import Session
from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client
from streamable_http_sigv4 import streamablehttp_client_with_sigv4

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)


def create_streamable_http_transport_sigv4(
    mcp_url: str, service_name: str, region: str
):
    """
    Create a streamable HTTP transport with AWS SigV4 authentication.

    This function creates an MCP client transport that uses AWS Signature Version 4 (SigV4)
    to authenticate requests. This is necessary because standard MCP clients don't natively
    support AWS IAM authentication, and this bridges that gap.

    Args:
        mcp_url (str): The URL of the MCP gateway endpoint
        service_name (str): The AWS service name for SigV4 signing (typically "bedrock-agentcore")
        region (str): The AWS region where the gateway is deployed

    Returns:
        StreamableHTTPTransportWithSigV4: A transport instance configured for SigV4 auth

    Example:
        >>> transport = create_streamable_http_transport_sigv4(
        ...     mcp_url=".../mcp",
        ...     service_name="bedrock-agentcore",
        ...     region="us-west-2"
        ... )
    """
    # Get AWS credentials from the current boto3 session
    # These credentials will be used to sign requests with SigV4
    session = boto3.Session()
    credentials = session.get_credentials()

    # Create and return the custom transport with SigV4 signing capability
    return streamablehttp_client_with_sigv4(
        url=mcp_url,
        credentials=credentials,
        service=service_name,
        region=region,
    )


def get_full_tools_list(client):
    """
    Retrieve the complete list of tools from an MCP client, handling pagination.

    MCP servers may return tools in paginated responses. This function handles the
    pagination automatically and returns all available tools in a single list.

    Args:
        client: An MCP client instance (from strands.tools.mcp.mcp_client.MCPClient)

    Returns:
        list: A complete list of all tools available from the MCP server

    Example:
        >>> mcp_client = MCPClient(lambda: create_transport())
        >>> all_tools = get_full_tools_list(mcp_client)
        >>> print(f"Found {len(all_tools)} tools")
    """
    more_tools = True
    tools = []
    pagination_token = None

    # Loop until we've fetched all pages
    while more_tools:
        tmp_tools = client.list_tools_sync(pagination_token=pagination_token)

        tools.extend(tmp_tools)

        # Check if there are more pages to fetch
        if tmp_tools.pagination_token is None:
            # No more pages - we're done
            more_tools = False
        else:
            # More pages exist - prepare to fetch the next one
            more_tools = True
            pagination_token = tmp_tools.pagination_token

    return tools


async def main():
    boto_session = Session()
    region = boto_session.region_name
    print(f"Using AWS region: {region}")

    ssm_client = boto3.client("ssm", region_name=region)

    agent_arn_response = ssm_client.get_parameter(
        Name="/mcp_server/runtime_iam/agent_arn"
    )
    agent_arn = agent_arn_response["Parameter"]["Value"]
    print(f"Retrieved Agent ARN: {agent_arn}")

    if not agent_arn:
        print("❌ Error: AGENT_ARN not found")
        sys.exit(1)

    encoded_arn = agent_arn.replace(":", "%3A").replace("/", "%2F")
    mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"

    try:
        async with create_streamable_http_transport_sigv4(
                mcp_url=mcp_url, service_name="bedrock-agentcore", region=region
        ) as (
            read_stream,
            write_stream,
            _,
        ):
            async with ClientSession(read_stream, write_stream) as session:
                print("\n🔄 Initializing MCP session...")
                await session.initialize()
                print("✓ MCP session initialized")

                print("\n🔄 Listing available tools...")
                tool_result = await session.list_tools()

                print("\n📋 Available MCP Tools:")
                print("=" * 50)
                for tool in tool_result.tools:
                    print(f"🔧 {tool.name}: {tool.description}")

                print("\n🧪 Testing MCP Tools:")
                print("=" * 50)

                try:
                    print("\n➕ Testing add_numbers(5, 3)...")
                    add_result = await session.call_tool(
                        name="add_numbers", arguments={"a": 5, "b": 3}
                    )
                    print(f"   Result: {add_result.content[0].text}")
                except Exception as e:
                    print(f"   Error: {e}")

                try:
                    print("\n✖️  Testing multiply_numbers(4, 7)...")
                    multiply_result = await session.call_tool(
                        name="multiply_numbers", arguments={"a": 4, "b": 7}
                    )
                    print(f"   Result: {multiply_result.content[0].text}")
                except Exception as e:
                    print(f"   Error: {e}")

                try:
                    print("\n👋 Testing greet_user('Alice')...")
                    greet_result = await session.call_tool(
                        name="greet_user", arguments={"name": "Alice"}
                    )
                    print(f"   Result: {greet_result.content[0].text}")
                except Exception as e:
                    print(f"   Error: {e}")

                print("\n✅ MCP tool testing completed!")

    except Exception as e:
        print(f"❌ Error connecting to MCP server: {e}")
        import traceback

        print("\n🔍 Full error traceback:")
        traceback.print_exc()
        sys.exit(1)


if __name__ == "__main__":
    asyncio.run(main())


## Test Tool Invocation

Let's test our MCP tools by actually invoking them:

In [ ]:
print("Testing MCP tool invocation...")
print("=" * 50)
!python invoke_mcp_tools.py

## AgentCore 런타임(Runtime)에 MCP 서버 시작

이제 도커 파일이 준비되었으니 MCP 서버를 AgentCore 런타임(Runtime)에 시작해 보겠습니다. 이렇게 하면 Amazon ECR 리포지토리와 AgentCore 런타임(Runtime)이 생성됩니다.

<div style="text-align:left">
    <img src="images/launch.png" width="85%"/>
</div>

## Cleanup (Optional)

If you want to clean up the resources created during this tutorial, run the following cells:

In [ ]:
# try:
#     ssm_client.delete_parameter(Name='/mcp_server/runtime_iam/agent_arn')
#     print("✓ Parameter Store parameter deleted")
# except ssm_client.exceptions.ParameterNotFound:
#     print("ℹ️  Parameter Store parameter not found")

In [ ]:
# destroy_bedrock_agentcore(
#     config_path=Path(".bedrock_agentcore.yaml"),
#     agent_name=tool_name,
#     delete_ecr_repo=True
# )

# 🎉 Congratulations!

You have successfully:

✅ **Created an MCP server** with custom tools  
✅ **Tested locally** using MCP client  
✅ **Set up authentication** with Amazon Cognito  
✅ **Deployed to AWS** using AgentCore Runtime  
✅ **Invoked remotely** with proper authentication  
✅ **Learned MCP concepts** and best practices  

Your MCP server is now running on Amazon Bedrock AgentCore Runtime and ready for production use!

## Summary

In this tutorial, you learned how to:
- Build MCP servers using FastMCP
- Configure stateless HTTP transport for AgentCore compatibility
- Set up AWS IAM inbound authentication
- Deploy and manage MCP servers on AWS
- Test both locally and remotely
- Use MCP clients for tool invocation

The deployed MCP server can now be integrated into larger AI applications and workflows!

## 원격 테스트 클라이언트 생성

이제 배포(Deployment)된 MCP 서버를 테스트할 클라이언트를 생성해 보겠습니다. 이 클라이언트는 AWS에서 필요한 자격 증명을 검색하고 배포(Deployment)된 서버에 연결합니다: